In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col, broadcast, collect_set, size, array_contains

class Transformer:
    def __init__(self):
        pass
    def transform(self,inputDF):
        pass
class FirstTransform(Transformer):
    def transform(self,inputDF):
        #cusomers who bought airpods after iphone
        transactioninputDF = inputDF.get("transactionDF")
        print("transaction inputDF in transform")
        #transactioninputDF.show()

        windowSpec = Window.partitionBy("customer_id").orderBy("transaction_date")


        transformeddf = transactioninputDF.withColumn("next_prod_name",lead("product_name").over(windowSpec) )
        print("airpods after iphone")
        filteredDF = transformeddf.filter(
            (col("product_name") == 'iPhone') & (col("next_prod_name") == 'AirPods')
        )
        

        customertable = inputDF.get("customerInputDF")
        
        joinDF = customertable.join(broadcast(filteredDF),"customer_id")
        
        print("new new final transformed DF")
        finalDF= joinDF.select(
            "customer_id",
            "customer_name",
            "location")
        return finalDF
class OnlyAirpodsAndIphone(Transformer):

    def transform(self, inputDF):
        """
        Customer who have bought only iPhone and Airpods nothing else
        """

        transactioninputDF = inputDF.get("transactionDF")
        print("transaction inputDF in transform")

        groupedDF = transactioninputDF.groupBy("customer_id").agg(
            collect_set("product_name").alias("products")
        )

        print("Grouped DF")
        groupedDF.show()

        filteredDF = groupedDF.filter(
            (array_contains(col("products"), "iPhone")) &
            (array_contains(col("products"), "AirPods")) & 
            (size(col("products")) == 2)
        )
        
        print("Only Airpods and iPhone")
        filteredDF.show()

        customerInputDF = inputDF.get("customerInputDF")

        customerInputDF.show()

        joinDF =  customerInputDF.join(
           broadcast(filteredDF),
            "customer_id"
        )

        print("JOINED DF")
        joinDF.show()

        return joinDF.select(
            "customer_id",
            "customer_name",
            "location"
        )

       
        